In [ ]:
### Make CSV of CONUS2 flow at matched gages ###
### DTT, 10/22

# This script is split into two main parts: 1) reading in CONUS2 gridded total (aggregated) daily flow for the full domain and creating a dataframe of CONUS2 flow for cells that have been matched with USGS gages in the `NWM_Gage_Adjustments_final.csv`. 2) matching the gages that both have flow between the PF csv and USGS csv retrieved from hydrodata. 
# Note that flow is converted in this script from daily accumulated flow in [m^3/h] to daily mean flow in cms or [m^3/s].

### Inputs:
# - `NWM_Gage_Adjustments_final.csv` - this can be found on the CONUS2 Dropbox or in /glade/p/univ/ucsm0002/CONUS2/domain_files
# - Daily total streamflow PFCLM outputs as PFBs - processed using `compute_daily_PF_averages.py`
# - USGS daily flow csv - from the hydrodata repository on Verde

### Outputs:
# - CSV of PFCLM daily mean flow (cms) with gage ID, lat/long, and CONUS2 cell location
# - two flow-matched CSVs for PF and USGS flow
# - note that the CSV outputs with 'day 001' which starts at the water year (001 == October 1)***

# Notes:
# - need to fix the no_days, currently this will only be accurate if this is started at the begninning of a water year and need to add in some dictionary or if statement to specify num days in a month or something like that.
# - ***need to change day headings so that they are more descriptive than 'day 001' and have an actual date

import sys
from parflow.tools.io import read_pfb,write_pfb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Directory where PF flow PFBs are saved in
directory_out = '/glade/scratch/tijerina/CONUS2/spinup_WY2003/averages'
organized_dir = '/glade/p/univ/ucsm0002/CONUS2/Validation/Streamflow/Organized_Daily_Flow'

usgs_data = 'USGS_WY2003_oct_mean_flow.csv' #csv of USGS flow from hydrodata

# need to change water year and number of days
water_year = 2003
no_days = 31 

### check gage locations for daily flow
NWM_gage_csv = pd.read_csv('/glade/p/univ/ucsm0002/CONUS2/domain_files/NWM_Gage_Adjustments_final.csv')


### set up pandas dataframe of gage ID, lat/long, CONUS2 x and y indices ###
pf_flow_df = pd.DataFrame(columns = ['STNID', 'USGS_lat', 'USGS_lon', 'x_new_adj', 'y_new_adj'])
pf_flow_df['STNID'] = NWM_gage_csv['STNID'].astype(int)
pf_flow_df['USGS_lat'] = NWM_gage_csv['USGS_lat']
pf_flow_df['USGS_lon'] = NWM_gage_csv['USGS_lon']
pf_flow_df['x_new_adj'] = NWM_gage_csv['x_new_adj']
pf_flow_df['y_new_adj'] = NWM_gage_csv['y_new_adj']

# add leading zeros to USGS gages
pf_flow_df['STNID'] = pf_flow_df['STNID'].astype('str').str.zfill(8)


### READ STREAMFLOW PFBs ###
# Read in CONUS2 daily streamflow PFBs and save as df in flow_df, convert to total accumulated in m^3/h to mean daily in cms
for i in range(no_days):
    step = str(int(i+1)).rjust(3, '0')
    pf_flow_pfb = np.squeeze(read_pfb(f'{directory_out}/flow.2003.daily.{step}.pfb'))
    pf_flow_df[f'day {step}'] = pf_flow_pfb[pf_flow_df['y_new_adj'],pf_flow_df['x_new_adj']]/3600/24 # CONVERT FROM m^3/h to cms AND from daily accumulated to daily mean
    print(f'reading flow for day {step} and converting from total accumulated flow in m^3/h, to daily mean flow in cms')

    
# Create column for matching/have flow (=1) and not matching/have no flow (=0) gages
pf_flow_df['matched'] = np.where(pf_flow_df['day 001']>0, 1, 0)

# remove cells with no flow and make new pandas df with matching flow at CONUS2 cells and USGS gages
pf_flow_df_NWM_match = pf_flow_df[pf_flow_df.matched != 0]

# SAVE OUT PANDAS DF FOR CONUS2 FLOW
### save csv file of all matching gage locations and CONUS2 daily flow, note the USGS STNID's drop the leading zeros when saving
pf_flow_df_NWM_match.to_csv(f'{organized_dir}/CONUS2_NWM_matched_flow_{water_year}.csv', sep = ",")

In [ ]:
###
### read in csv that was just created of gage locations and CONUS2 flow
pf_flow_df = pd.read_csv(f'{organized_dir}/CONUS2_NWM_matched_flow_{water_year}.csv',index_col=['STNID'])
pf_flow_df = pf_flow_df.drop(columns=['Unnamed: 0','USGS_lat','USGS_lon','x_new_adj','y_new_adj','matched']) #drop unnecessary columns for now
pf_flow_df.index.names = ['site_id']
                          
### read in USGS flow for all gages with flow in October
# streamflow from hydrodata has already been converted to cms!!!
usgs_flow_df = pd.read_csv(f'{organized_dir}/{usgs_data}',index_col=['site_id']) 
usgs_flow_df = usgs_flow_df.drop(columns=['Unnamed: 0', 'num_obs'])
                          
# merge the CONUS2 and USGS dataframes so that we remove all gage locations that don't match between the two
combine_df = pf_flow_df.merge(usgs_flow_df, on='site_id', how='inner')
                    
#********* FIX these next lines, the dates are hardcoded in ************
# separate the CONUS2 flow from the USGS flow in the combine_df
pf_flow_matched = pd.DataFrame(combine_df.iloc[:,0:31]) # get only CONUS2 flow
usgs_flow_matched = pd.DataFrame(combine_df.iloc[:,31:62]) #get only USGS flow 
                          
# Save Flow Matched CSVs
pf_flow_matched.to_csv(f'{organized_dir}/CONUS2_daily_FlowMatch_cms_{water_year}.csv', sep = ",")
usgs_flow_matched.to_csv(f'{organized_dir}/USGS_daily_FlowMatch_cms_{water_year}.csv', sep = ",")